In [2]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
import tensorflow as tf
import tqdm
import os

In [184]:
path = os.path.join("C:\\Users\\Robin\\Desktop\\deep_learning")
label = np.load(path+"\\processed_data\\v3_absolute_grape_changeling-12_2293-4124NDTRAIN.npy", allow_pickle=True)[0]
image = np.load(path+"\\processed_data\\v3_absolute_grape_changeling-12_2293-4124NDTRAIN.npy", allow_pickle=True)[1]
original_label = np.load(path+"\\MineRLNavigateDense-v0\\v3_absolute_grape_changeling-12_2293-4124\\NDTRAIN.npy", allow_pickle=True)[0]
og_label = np.load(path+"\\MineRLNavigateDense-v0\\v3_absolute_grape_changeling-12_2293-4124\\rendered.npz", allow_pickle=True)
#print(label)
#print(len(label))
data_params = ['reward', 'observation$compassAngle', 'observation$inventory$dirt', 'action$forward', 'action$left', 'action$back', 'action$right', 'action$jump', 'action$sneak', 'action$sprint', 'action$attack', 'action$camera', 'action$place']

print([len(og_label[i]) for i in data_params[3:]])
print(label.shape)
print(label)
actual_frames = len(og_label["reward"])
#image = image[-actual_frames:]
print(image.shape)


file_names = [folder_name for folder_name in os.listdir(path+"\\MineRLNavigateDense-v0")]

train_data = file_names[:160]
valid_data = file_names[160:190]

#idx = 0

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, batch_size):
        self.vid_num = 0
        self.last_frame = 0
        self.batch_size = batch_size
        self.data = data
        self.video_file_no = 0

    def __len__(self):
        return (np.ceil(len(self.data) / float(self.batch_size))).astype(np.int)
    
    '''def __next__(self):
        return self.getitem()'''

    def __getitem__(self, idx):
        #batch_x = self.data[idx * self.batch_size : (idx+1) * self.batch_size]
        video = self.data[self.video_file_no]
        
        image_part = np.array([np.load(path+"\\MineRLNavigateDense-v0\\"+video+"\\NDTRAIN.npy", allow_pickle=True)[0]])[idx * self.batch_size : (idx+1) * self.batch_size]
        label_part = np.array([np.load(path+"\\MineRLNavigateDense-v0\\"+video+"\\NDTRAIN.npy", allow_pickle=True)[1]])[idx * self.batch_size : (idx+1) * self.batch_size]
        self.last_frame += 1
        print("Images:")
        print(image_part[0])
        print("Labels:")
        print(label_part[0][0])
        print("Label shape: ", label_part[0][0].shape)
        #shape_o = int(np.array(image_part[0]).shape[0]/12288)
        print(shape_o)
        image_part = np.array(image_part[0]).reshape(10, 64, 64, 3)[-label_part[0][0].shape[0]:][idx * self.batch_size : (idx+1) * self.batch_size]
        print("Image shape: ",image_part.shape)
        #print(image_part, label_part[0])
        if len(self.data) != self.video_file_no:
            self.video_file_no += 1
        return image_part, label_part
    
batch_size = 10

training_gen = DataGenerator(train_data, batch_size)
valid_gen = DataGenerator(valid_data, batch_size)

#dg = DataGenerator(file_names, batch_size)
#for x in range(2):
    #plt.imshow(np.array(next(dg))[0].reshape(1792, 64, 64, 3).astype(np.uint8)[50])
    #plt.imshow(np.array(next(dg))[0].astype(np.uint8)[10])
    #plt.show()

#print(DataGenerator())
print("Shape of training data: ", np.array(training_gen).shape)
print("Shape of valid data: ", np.array(valid_gen).shape) 

[1792, 1792, 1792, 1792, 1792, 1792, 1792, 1792, 1792, 1792]
(16129,)
[0 0 0 ... 0 0 0]
(1832, 64, 64, 3)
Images:
[109. 133. 193. ... 142. 164. 205.]
Labels:
[ 0.0000000e+00  6.3092217e-02  2.1552046e-01 ... -8.5000791e-02
 -8.9136705e-02  9.9913994e+01]
Label shape:  (1792,)
1832
Image shape:  (10, 64, 64, 3)
Images:
Images:
[104. 129. 182. ...  70.  91.  41.]
Labels:
[ 1.08652599e-02  1.16004460e-02  1.55219417e-02  1.90406647e-02
  2.21165475e-02  2.47519109e-02  2.68445555e-02  2.85083521e-02
  2.08570249e-02  9.68514010e-03 -1.74363062e-03 -7.89845269e-03
  0.00000000e+00 -2.25815270e-02 -1.16575966e-02 -1.09297950e-02
  0.00000000e+00 -1.89451240e-02 -7.98203610e-03 -6.97072595e-03
 -5.50712645e-03 -4.34810435e-03 -3.45034222e-03  0.00000000e+00
 -3.23355268e-03 -8.63500987e-04  0.00000000e+00  7.48518971e-04
  2.85884459e-03  5.09207090e-03  7.19714444e-03  9.17980354e-03
  1.14341341e-02  1.36454245e-02  1.57571901e-02  1.83313601e-02
  2.10918523e-02  2.37893220e-02  2.9952939

Image shape:  (10, 64, 64, 3)
Images:


IndexError: index 0 is out of bounds for axis 0 with size 0

In [151]:
conv = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(32, 5, activation="relu", padding="same", input_shape=[64, 64, 3]),
    tf.keras.layers.MaxPooling2D((2,2), padding="same"),
    tf.keras.layers.Conv2D(64, 5, activation="relu", padding="same"),
    tf.keras.layers.MaxPooling2D((2,2), padding="same"),
    tf.keras.layers.Conv2D(64, 5, activation="relu", padding="same"),
    tf.keras.layers.MaxPooling2D((2,2), padding="same"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")

])
    
conv.compile(loss="mse", optimizer="adam", metrics="accuracy")
    
conv.fit_generator(generator = training_gen,
                    steps_per_epoch = int(3800 // batch_size),
                    epochs = 10,
                    verbose = 1,
                    validation_data = valid_gen,
                    validation_steps = int(950 // batch_size))



1793
1832
(1793, 64, 64, 3)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [88]:
training_data_path = "C:\\Users\\Robin\\Desktop\\deep_learning\\MineRLNavigateDense-v0"

def preprocess_data():
    training_valid_testing = [100, 50, 43]
    #to do: split the video frame and y data into even smaller data
    for folder in tqdm.tqdm(os.listdir(training_data_path)):
        for each in range(len(training_valid_testing)): 
            for j in range(each):
                path = os.path.join(training_data_path+"\\"+folder)
                training_data = np.load(path+"\\NDTRAIN.npy", allow_pickle=True)
                training_data[0] = np.array(training_data[0]).reshape(int(training_data[0].shape[0]/12288), 64, 64, 3)
                print(np.array(training_data[1])[-2][0])
                training_data[1] = np.concatenate(np.array(training_data[1]))
                print(training_data[1].shape)

                clustering = KMeans(n_clusters=32, random_state=0).fit_predict(training_data[1])

preprocess_data()

  0%|                                                  | 0/193 [00:00<?, ?it/s]

[0. 0.]


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 11 has 2 dimension(s)